Case study: 311 service requests.

Prompt: Please describe the steps you take to ingest and process the data, what are the challenges and how you overcome them.

Aggregate the 311 records in ways that best describes the characteristics of the inbound call patterns. Explain and visualize your findings. What are the implications you can infer from the patterns?


Final goal of project:predict the daily 311 inbound calls for the next 7 days 

•	Use your insights from previous sections to build a predictive model of your choice – explain the assumptions of your model and why you picked it.

•	What features are you using in your model and, how does your data exploration process help you decide which features you are incorporating in your model?

•	Evaluate the impact of weather on the call volume, find patterns and define causal impact if there’s any. 

•	Build a reasonable model framework, explain your model results, and justify the relationships identified in the model. Not necessary to dig into complex models.


In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

Import data. Note that this takes a couple of minutes due to the large file size.

In [3]:
c_path = '311-2016-2018.csv'
c = pd.read_csv(c_path, low_memory=False)

Get an initial impression of the table we are working with.

In [4]:
c.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location,Created Year
0,38237851,01/17/2018 02:39:00 PM,01/24/2018 12:00:00 AM,DSNY,Department of Sanitation,Request Large Bulky Item Collection,Request Large Bulky Item Collection,Sidewalk,11222,95 GREENPOINT AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,40.730013,-73.956267,"(40.73001299919553, -73.95626650502489)",2018.0
1,38237852,01/17/2018 01:40:00 PM,01/21/2018 12:00:00 AM,DSNY,Department of Sanitation,Request Large Bulky Item Collection,Request Large Bulky Item Collection,Sidewalk,10308,40 ASHWOOD COURT,...,NaN,NaN,NaN,NaN,NaN,NaN,40.558494,-74.141962,"(40.558494382393974, -74.14196245468202)",2018.0
2,38237855,01/17/2018 10:42:00 PM,01/20/2018 10:02:00 PM,DSNY,Department of Sanitation,Request Large Bulky Item Collection,Request Large Bulky Item Collection,Sidewalk,11361,204-36 47 ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,40.754075,-73.775594,"(40.75407455058778, -73.77559379608053)",2018.0
3,38237856,01/17/2018 07:26:00 PM,01/19/2018 12:00:00 AM,DSNY,Department of Sanitation,Request Large Bulky Item Collection,Request Large Bulky Item Collection,Sidewalk,10467,3851 BARNES AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,40.884181,-73.859111,"(40.88418051177941, -73.85911054646147)",2018.0
4,38237857,01/17/2018 07:13:00 PM,01/20/2018 07:41:00 PM,DSNY,Department of Sanitation,Request Large Bulky Item Collection,Request Large Bulky Item Collection,Sidewalk,11215,160 7 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,40.672113,-73.977242,"(40.67211272839247, -73.97724198791173)",2018.0


Look at a single row to see what columns exist and what data types there are. Many entries are NaN, which is likely going to be true quite often.

In [5]:
c.iloc[0,:]

Unique Key                                                       38237851
Created Date                                       01/17/2018 02:39:00 PM
Closed Date                                        01/24/2018 12:00:00 AM
Agency                                                               DSNY
Agency Name                                      Department of Sanitation
Complaint Type                        Request Large Bulky Item Collection
Descriptor                            Request Large Bulky Item Collection
Location Type                                                    Sidewalk
Incident Zip                                                        11222
Incident Address                                     95 GREENPOINT AVENUE
Street Name                                             GREENPOINT AVENUE
Cross Street 1                                            FRANKLIN STREET
Cross Street 2                                           MANHATTAN AVENUE
Intersection Street 1                 

In [68]:
times = c.loc[0:10,'Created Date'].astype('datetime64')

In [69]:
times

0    2018-01-17 14:39:00
1    2018-01-17 13:40:00
2    2018-01-17 22:42:00
3    2018-01-17 19:26:00
4    2018-01-17 19:13:00
5    2018-01-17 18:58:00
6    2018-01-17 17:01:00
7    2018-01-17 15:28:00
8    2018-01-17 14:30:00
9    2018-01-17 14:29:00
10   2018-01-17 12:54:00
Name: Created Date, dtype: datetime64[ns]

Above method seems reasonable for converting date to datetime. Now need to get comparison working>

In [66]:
c.loc[0:10, 'Created Date']

0     01/17/2018 02:39:00 PM
1     01/17/2018 01:40:00 PM
2     01/17/2018 10:42:00 PM
3     01/17/2018 07:26:00 PM
4     01/17/2018 07:13:00 PM
5     01/17/2018 06:58:00 PM
6     01/17/2018 05:01:00 PM
7     01/17/2018 03:28:00 PM
8     01/17/2018 02:30:00 PM
9     01/17/2018 02:29:00 PM
10    01/17/2018 12:54:00 PM
Name: Created Date, dtype: object

In [60]:
c.dtypes

Unique Key                          int64
Created Date                       object
Closed Date                        object
Agency                             object
Agency Name                        object
Complaint Type                     object
Descriptor                         object
Location Type                      object
Incident Zip                       object
Incident Address                   object
Street Name                        object
Cross Street 1                     object
Cross Street 2                     object
Intersection Street 1              object
Intersection Street 2              object
Address Type                       object
City                               object
Landmark                           object
Facility Type                      object
Status                             object
Due Date                           object
Resolution Description             object
Resolution Action Updated Date     object
Community Board                   

In [58]:
c.loc[:10, 'Created Date'].to_numpy()

array(['01/17/2018 02:39:00 PM', '01/17/2018 01:40:00 PM',
       '01/17/2018 10:42:00 PM', '01/17/2018 07:26:00 PM',
       '01/17/2018 07:13:00 PM', '01/17/2018 06:58:00 PM',
       '01/17/2018 05:01:00 PM', '01/17/2018 03:28:00 PM',
       '01/17/2018 02:30:00 PM', '01/17/2018 02:29:00 PM',
       '01/17/2018 12:54:00 PM'], dtype=object)

In [48]:
c.loc[:10, 'Created Date'][0][0:10]

'01/17/2018'

In [26]:
c.loc[:20, ['Created Date', 'Closed Date', 'Agency Name', 'Complaint Type']]

,Created Date,Closed Date,Agency Name,Complaint Type
0,01/17/2018 02:39:00 PM,01/24/2018 12:00:00 AM,Department of Sanitation,Request Large Bulky Item Collection
1,01/17/2018 01:40:00 PM,01/21/2018 12:00:00 AM,Department of Sanitation,Request Large Bulky Item Collection
2,01/17/2018 10:42:00 PM,01/20/2018 10:02:00 PM,Department of Sanitation,Request Large Bulky Item Collection
3,01/17/2018 07:26:00 PM,01/19/2018 12:00:00 AM,Department of Sanitation,Request Large Bulky Item Collection
4,01/17/2018 07:13:00 PM,01/20/2018 07:41:00 PM,Department of Sanitation,Request Large Bulky Item Collection
5,01/17/2018 06:58:00 PM,01/18/2018 10:00:00 PM,Department of Sanitation,Request Large Bulky Item Collection
6,01/17/2018 05:01:00 PM,01/21/2018 12:00:00 AM,Department of Sanitation,Request Large Bulky Item Collection
7,01/17/2018 03:28:00 PM,01/21/2018 12:00:00 AM,Department of Sanitation,Request Large Bulky Item Collection
8,01/17/2018 02:30:00 PM,01/23/2018 12:00:00 AM,Department of Sanitation,Request Large Bulky Item Collection
9,01/17/2018 02:29:00 PM,01/19/2018 08:34:00 PM,Department of Sanitation,Request Large Bulky Item Collection


Explore different complaint types that exist.

In [6]:
c.loc[100:125,'Complaint Type']

100              Noise - Residential
101              Noise - Residential
102              Noise - Residential
103              Noise - Residential
104              Noise - Residential
105              Noise - Residential
106                            Noise
107      Non-Emergency Police Matter
108                      Air Quality
109                             Lead
110                             Lead
111                     Water System
112                    Drug Activity
113                            Noise
114                            Noise
115                 Street Condition
116                            Noise
117             Sanitation Condition
118             Sanitation Condition
119                            Sewer
120                            Sewer
121    Root/Sewer/Sidewalk Condition
122          Overgrown Tree/Branches
123               Water Conservation
124               Water Conservation
125               Consumer Complaint
Name: Complaint Type, dtype: object

Check extent of locations (latitude and longitude). Appears to be only NYC (which is different from the weather, which has weather from all over the state).

In [7]:
print(np.min(c.loc[:,'Latitude']))
print(np.max(c.loc[:,'Latitude']))
print(np.min(c.loc[:,'Longitude']))
print(np.max(c.loc[:,'Longitude']))

40.49804421521046
40.91294056699566
-74.25521082506387
-73.70038354802529


Explore what types of complaints are present.

In [12]:
complaint_types = np.unique(c.loc[:,'Complaint Type'])
print(complaint_types)
print('Number of unique complaint types: ' , len(unique_complaints))

['AGENCY' 'APPLIANCE' 'ATF' 'Abandoned Vehicle' 'Adopt-A-Basket'
 'Advocate - Levy' 'Advocate - Lien' 'Advocate - Other' 'Advocate - RPIE'
 'Advocate-Business Tax' 'Advocate-Co-opCondo Abatement'
 'Advocate-Commercial Exemptions' 'Advocate-Personal Exemptions'
 'Advocate-Prop Class Incorrect' 'Advocate-Prop Refunds/Credits'
 'Advocate-Property Value' 'Advocate-SCRIE/DRIE' 'Advocate-UBT'
 'Air Quality' "Alzheimer's Care" 'Animal Abuse'
 'Animal Facility - No Permit' 'Animal in a Park' 'Animal-Abuse'
 'Asbestos' 'Asbestos/Garbage Nuisance' 'BEST/Site Safety'
 'Beach/Pool/Sauna Complaint' 'Benefit Card Replacement'
 'Bereavement Support Group' 'Bike Rack Condition'
 'Bike/Roller/Skate Chronic' 'Blocked Driveway' 'Boilers' 'Borough Office'
 'Bottled Water' 'Bridge Condition' 'Broken Muni Meter'
 'Broken Parking Meter' 'Building Condition' 'Building Marshals office'
 'Building/Use' 'Bus Stop Shelter Complaint' 'Bus Stop Shelter Placement'
 'Calorie Labeling' 'Case Management Agency Complain

Explore frequency of complaints over time. (Using Created date)

In [13]:
c.loc[:5, 'Created Date']

0    01/17/2018 02:39:00 PM
1    01/17/2018 01:40:00 PM
2    01/17/2018 10:42:00 PM
3    01/17/2018 07:26:00 PM
4    01/17/2018 07:13:00 PM
5    01/17/2018 06:58:00 PM
Name: Created Date, dtype: object